### Weird XY_power_sum_E jump every once in a while, and weird XY_power_imbalance_E


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_csv(r"C:\ACME_analysis\Pointing0014.0109\parity_transformed_block_df.csv")

In [ ]:
px.scatter(y = df['XY_power_sum_E'], color = 2 * (df['P'] + 1) + 1 + df['R'])

### Conclusion is the spread of XY_power_sum is grouped by PR switch combinations.

In [26]:
px.scatter(y = df['XY_power_imbalance_E'], color = 2 * (df['P'] + 1) + 1 + df['R'])

In [25]:
px.scatter(y = np.abs(df['XY_power_imbalance_E']), color = 2 * (df['P'] + 1) + 1 + df['R'])

## Examination of whether E corrlation label is real or not (from drifting + switch sequence randomized order)

In [28]:
df_full = pd.read_csv(r"C:\ACME_analysis\Pointing0014.0109\purged_aggregated_header.csv")

In [31]:
E_begin = df_full.groupby(['block']).first().reset_index()['E']

In [32]:
df['E_begin'] = E_begin

In [36]:
df_snippet = df[(df['P'] == + 1) & (df['R'] == - 1)]
px.scatter(x = df_snippet['block'], y = df_snippet['XY_power_sum_E'], color = df_snippet['E_begin'])

In [37]:
df_snippet = df[(df['P'] == - 1) & (df['R'] == - 1)]
px.scatter(x = df_snippet['block'], y = df_snippet['XY_power_sum_E'], color = df_snippet['E_begin'])

In [39]:
df_snippet = df[(df['P'] == - 1) & (df['R'] == + 1)]
px.scatter(x = df_snippet['block'], y = df_snippet['XY_power_sum_E'], color = df_snippet['E_begin'])

In [40]:
df_snippet = df[(df['P'] == + 1) & (df['R'] == + 1)]
px.scatter(x = df_snippet['block'], y = df_snippet['XY_power_sum_E'], color = df_snippet['E_begin'])

### Same examination with XY_power_diff

In [42]:
df_snippet = df[(df['P'] == - 1) & (df['R'] == + 1)]
px.scatter(x = df_snippet['block'], y = df_snippet['XY_power_imbalance_E'], color = df_snippet['E_begin'])

Does not seems to be correlated with E_begin switch. Consider populating the first switches

In [47]:
for switch in ['N', 'E','Q' , 'B', 'P', 'R']:
    try:
        df[switch + '_begin'] = df_full.groupby(['block']).first().reset_index()[switch]
    except:
        continue


In [52]:
df_snippet = df[(df['P'] == - 1) & (df['R'] == + 1)]
px.scatter(x = df_snippet['block'], y = df_snippet['XY_power_imbalance_E'], color = df_snippet['N_begin'] * df_snippet['E_begin'])

# OK, so there is some weird things about N happening here.... NE order goes into E channel...

In [54]:
px.scatter(df['XY_power_imbalance_N'], color = df['P'] * 2 + df['R'] + 1)

In [55]:
px.scatter(df['XY_power_imbalance_NE'], color = df['P'] * 2 + df['R'] + 1)

In [ ]:
df_full['NE'] = df_full['N'] * df_full['E']


In [95]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create shared x-axis subplots
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=('XY Power Sum', 'XY Power Imbalance')
)

x_vals = df_full['block']*16 + df_full['trace']
color_vals = df_full['P'] * 2 + df_full['R'] + 1

# Trace for XY_power_sum
trace0 = go.Scatter(x=x_vals, y=df_full['XY_power_sum'],
                    marker=dict(color=color_vals),
                    mode='lines+markers', name='XY_power_sum')

# Trace for XY_power_imbalance
trace1 = go.Scatter(x=x_vals, y=df_full['XY_power_imbalance'],
                    marker=dict(color=color_vals),
                    mode='lines+markers', name='XY_power_imbalance')

# Visual helper traces
trace_N = go.Scatter(x=x_vals, y=df_full['N'],
                     mode='lines', line_shape='hvh', name='N', visible='legendonly')
trace_E = go.Scatter(x=x_vals, y=df_full['E'],
                     mode='lines', line_shape='hvh', name='E', visible='legendonly')
trace_NE = go.Scatter(x=x_vals, y=df_full['NE'],
                      mode='lines', line_shape='hvh', name='NE', visible='legendonly')

# Add traces to each subplot
fig.add_trace(trace0, row=1, col=1)
fig.add_trace(trace_N, row=1, col=1)
fig.add_trace(trace_E, row=1, col=1)
fig.add_trace(trace_NE, row=1, col=1)

fig.add_trace(trace1, row=2, col=1)
fig.add_trace(trace_N, row=2, col=1)
fig.add_trace(trace_E, row=2, col=1)
fig.add_trace(trace_NE, row=2, col=1)

# Y-axis ranges
fig.update_yaxes(range=[0.75, 0.98], row=1, col=1)
fig.update_yaxes(range=[0.96, 0.98], row=2, col=1)

# Layout update
fig.update_layout(height=600, showlegend=True)

fig.show()

Observations:

- 1. P state modulates the XY imbalance by 2E-3 level (not stable, everchanging, there is period without any modulation)
- 2. N state also modulates the XY imbalance by 2E-3 level (not stable, everchanging, there is also period without any modulation, around trace 2000, and 2900 - 3000. It also seems to have minimal P modulation? So when P difference is small, N oscillating effect is also small?)
- 3. It seems the occurance of XY power imbalance being stable coincide with certain P+ P- relative power in XY power sum.

In [104]:
px.line(df['XY_power_imbalance_N'])

In [100]:
px.line(df['E_begin'] * df['N_begin'])